In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.contrib import rnn
import pprint
pp = pprint.PrettyPrinter(indent=4)
sess = tf.InteractiveSession()

In [2]:
# RNN model
#rnn_cell - rnn_cell.BasicRNNCell(rnn_size)

#outputs, _states = tf.nn.dynamic_rnn(rnn_cell,
#                                   X,
#                                   initial_state=initial_state,
#                                   dtype=tf.float32)

hidden_size = 5
input_dim = 5
batch_size = 1
sequence_length = 6

In [3]:
idx2char = ['h','i','e','l','o']
x_data = [[0,1,0,2,3,3]]
x_one_hot = [[[1,0,0,0,0],
             [0,1,0,0,0],
             [1,0,0,0,0],
             [0,0,1,0,0],
             [0,0,0,1,0],
             [0,0,0,1,0]]]

y_data = [[1,0,2,3,3,4]]

X = tf.placeholder(
    tf.float32, [None, sequence_length, input_dim]) # X one-hot
Y = tf.placeholder(
    tf.int32, [None, sequence_length]) # Y label

In [4]:
cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size, state_is_tuple=True)
initial_state = cell.zero_state(batch_size, tf.float32)
outputs, _states = tf.nn.dynamic_rnn(
    cell, X, initial_state=initial_state, dtype=tf.float32)

In [5]:
#
y_data = tf.constant([[1,1,1]])

#
prediction1 = tf.constant([[ [0.3, 0.7], [0.3, 0.7], [0.3, 0.7] ]], dtype=tf.float32)
prediction2 = tf.constant([[ [0.1, 0.9], [0.1, 0.9], [0.1, 0.9] ]], dtype=tf.float32)

#
weights = tf.constant([[1,1,1]], dtype=tf.float32)

sequence_loss1 = tf.contrib.seq2seq.sequence_loss(
    logits=prediction1, targets=y_data, weights=weights)
sequence_loss2 = tf.contrib.seq2seq.sequence_loss(
    logits=prediction2, targets=y_data, weights=weights)

sess.run(tf.global_variables_initializer())
print("Loss1 : ", sequence_loss1.eval())
print("Loss2 : ", sequence_loss2.eval())

Loss1 :  0.513015
Loss2 :  0.371101


In [6]:
weights = tf.ones([batch_size, sequence_length])

sequence_loss = tf.contrib.seq2seq.sequence_loss(
    logits=outputs, targets=Y, weights=weights)
loss = tf.reduce_mean(sequence_loss)
train = tf.train.AdamOptimizer(learning_rate=0.1).minimize(loss)

In [7]:
prediction = tf.argmax(outputs, axis=2)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(2000):
        l, _ = sess.run([loss, train], feed_dict={X: x_one_hot, Y: y_data})
        result = sess.run(prediction, feed_dict={X: x_one_hot})
        print(i, "loss: ", l, "prediction: ", result, "true Y: ", y_data)
        
        # print char using dic
        result_str = [idx2char[c] for c in np.squeeze(result)]
        print("\tPrediction str : ", ''.join(result_str))

TypeError: The value of a feed cannot be a tf.Tensor object. Acceptable feed values include Python scalars, strings, lists, numpy ndarrays, or TensorHandles.For reference, the tensor object was Tensor("Const:0", shape=(1, 3), dtype=int32) which was passed to the feed with key Tensor("Placeholder_1:0", shape=(?, 6), dtype=int32).